In [1]:
#  9: strict=True, contrib=True, dominance=True      *DHMZ
# 10: strict=False, contrib=True, dominance=True     *DHM
# 11: strict=False, contrib=True, dominance=False    *CHM
# 12: strict=False, contrib=False, dominance=False   *VEM
# 13: strict=True, contrib=False, dominance=True     *SHMZ
# 14: strict=False, contrib=False, dominance=True    *SHM

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import imageio

In [3]:
# defining constants
N=80
ro=0.5
u=0.1
r_rep=0.1
r_adh=1.1
c_align=1.
c_rep=0.1
c_adh=0.0002
r=1.1
T=200

L=np.sqrt(N/(0.6*ro))

eta=1
strict=False
contrib=False
dominance=True
q = 2
dt = 1

In [4]:
# initializing locations and directions and noise
x=np.random.uniform(low=0.0, high=L, size=N)
y=np.random.uniform(low=0.0, high=0.6*L, size=N)
theta=np.random.uniform(low=-np.pi, high=np.pi, size=N)
noise=np.random.uniform(low=-eta/2, high=eta/2, size=T)

In [5]:
# madking the contribution matrix and dominance matrix
if contrib==True:
    C = np.random.lognormal(mean=0.0, sigma=1.0, size=N**2).reshape((N,N))

elif contrib==False:
    C = q*np.ones(N**2).reshape((N,N))

if dominance==True:
    B=np.tril(np.ones(int(N)))
    if strict:
        for i in range(N):
            B[i,i]=0 
            
elif dominance==False:
    B=np.ones((N,N))

In [6]:
# making an array containing locations and directions during time
data=np.zeros((N,5,T))
data[:,0,0]=x
data[:,1,0]=y
data[:,2,0]=theta
data[:,3,0]=u*np.cos(theta)
data[:,4,0]=u*np.sin(theta)

In [7]:
# update adjacency and distance matrices
def update_A(data, t):
    A = np.zeros((N,N))
    distances = np.zeros((N,N))
    x_distances = np.zeros((N,N))
    y_distances = np.zeros((N,N))
    for i in range(N):
        for j in range(i,N):

            dx=data[i,0,t]-data[j,0,t]
            dy=data[i,1,t]-data[j,1,t]
            distance = np.sqrt( dx**2 + dy**2 )
            if distance <= r:
                A[i,j] = 1
                A[j,i] = 1
            distances[i,j] = distance
            distances[j,i] = distance
            x_distances[i,j] = dx
            x_distances[j,i] = dx
            y_distances[i,j] = dy
            y_distances[j,i] = dy
            
    return A, distances, x_distances, y_distances

In [8]:
# alignment velocity
def update_align(data, t):
    mean_theta = np.arctan2(np.matmul(l,np.sin(data[:,2,t])),np.matmul(l,np.cos(data[:,2,t])))
    theta_align = mean_theta + noise[t]

    v_align_x = c_align*u*np.cos(theta_align)
    v_align_y = c_align*u*np.sin(theta_align)
    
    return v_align_x, v_align_y

In [9]:
# repulsion velocity
def update_rep(data, t):
    temp = np.maximum(0,r_rep-distances)
    temp=temp/distances
    temp[np.isnan(temp) | np.isinf(temp)] = 0
    v_rep_x = c_rep * np.sum(l*x_distances*temp/r_rep,axis=1)
    v_rep_y = c_rep * np.sum(l*y_distances*temp/r_rep,axis=1)
    
    return v_rep_x, v_rep_y

In [10]:
# attraction velocity
def update_adh(data, t):
    temp=np.minimum(0,r_rep-distances)*(distances<r_adh)
    temp=temp/distances
    temp[np.isnan(temp) | np.isinf(temp)] = 0

    v_adh_x = c_adh * np.sum(l*x_distances*temp/(r_adh-r_rep),axis=1)
    v_adh_y = c_adh * np.sum(l*y_distances*temp/(r_adh-r_rep),axis=1)
    
    return v_adh_x, v_adh_y

In [11]:
for t in tqdm(range(T-1)):
    
    # update adjacency and distance matrices
    A, distances, x_distances, y_distances = update_A(data, t)
    
    # makeing the L matrix
    l = np.multiply(C, B) 
    l = np.multiply(l,A)
    
    # update alignment velocity
    v_align_x, v_align_y = update_align(data, t)
    # update repulsion velocity
    v_rep_x, v_rep_y = update_rep(data, t)    
    # update attraction velocity
    v_adh_x, v_adh_y = update_adh(data, t)    
    
    # update velocity and location
    data[:,3,t+1] = v_align_x +v_rep_x+v_adh_x
    data[:,4,t+1] = v_align_y +v_rep_y+v_adh_y
    data[:,2,t+1] = np.arctan2(data[:,4,t+1],data[:,3,t+1])
    

    data[:,0,t+1] = (data[:,0,t]+dt*u*np.cos(data[:,2,t]))
    data[:,1,t+1] = (data[:,1,t]+dt*u*np.sin(data[:,2,t]))


  0%|          | 0/199 [00:00<?, ?it/s]<ipython-input-9-9ed86812fe50>:4: RuntimeWarning: divide by zero encountered in true_divide
  temp=temp/distances
<ipython-input-10-f921ea0e3b60>:4: RuntimeWarning: invalid value encountered in true_divide
  temp=temp/distances
100%|██████████| 199/199 [00:02<00:00, 79.12it/s]


In [12]:
## making animation

In [13]:
def make_frame(x,y,vx,vy,name):
    fig, ax = plt.subplots(figsize=(15,0.6*15)) # note we must use plt.subplots, not plt.subplot

    ax.scatter(x,y,linewidths=5)

    for i in range(N):
        ax.arrow(x[i],y[i],7*vx[i],7*vy[i],zorder=0, head_length=0.06, head_width=0.06)

    ax.scatter(x[i],y[i],color='r',linewidths=7)
#     neighborhood = plt.Circle((x[i], y[i]), r,  fill=False,color='g')
#     ax.add_artist(neighborhood)

    plt.xlim(-L,2*L)
    plt.ylim(-0.6*L,2*0.6*L)

    plt.xticks([])
    plt.yticks([])
#     plt.show()
    plt.savefig(name+'.jpg')
    plt.close(fig)

In [14]:
image_path='./images14/'
for tt in tqdm(range(T)):
    make_frame(data[:,0,tt],data[:,1,tt],data[:,3,tt],data[:,4,tt],image_path+str(tt))

100%|██████████| 200/200 [00:25<00:00,  7.94it/s]


In [15]:
images = []
for t in range(T):
    images.append(imageio.imread(image_path+str(t)+'.jpg'))
imageio.mimsave('./movie14.gif', images,duration=0.05)